In [38]:
# !pip install textblob
# !pip install wordcloud
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from textblob import TextBlob
from IPython.display import Markdown, display
import os
import random
from tqdm import tqdm
from collections import defaultdict

from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

%matplotlib inline

In [4]:
data_final = pd.read_csv("preprocessed-file.csv")

In [29]:
data_final.shape

(16992, 3)

In [5]:
np.random.seed(50)

In [24]:
data_final.head()

,tweet_text,tag,text_final
0,"['am', 'i', 'mentally', 'ill', 'or', 'is', 'we...",-1,"['mentally', 'ill', 'wear', 'hair', 'tie', 'or..."
1,"['bath', 'salts', 'are', 'used', 'to', 'treat'...",-1,"['bath', 'salt', 'use', 'treat', 'mental', 'ph..."
2,"['insomnia', 'and', 'anxiety', 'really', 'pull...",-1,"['insomnia', 'anxiety', 'really', 'pull', 'bea..."
3,"['hab', 'leider', 'n', 'social', 'anxiety']",-1,"['hab', 'leider', 'n', 'social', 'anxiety']"
4,"['i', 'go', 'to', 'the', 'tip', 'yesterday', '...",-1,"['go', 'tip', 'yesterday', 'jolly', 'ikea', 'g..."


# **Tf-idf**

In [61]:


X = data_final['text_final']
Y = data_final['tag']
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(X)
X = Tfidf_vect.transform(X)

Encoder = LabelEncoder()
Y = Encoder.fit_transform(Y)

###  **1 . K Fold Naive Bayes**

In [62]:
# evaluate a Naive Bayes model using k-fold cross-validation
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

naive_model = naive_bayes.MultinomialNB()
# evaluate model
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores)*100, std(scores)))

Accuracy: 81.020 (0.008)


### **2. K Fold SVM**

In [63]:
# evaluate a SVM model using k-fold cross-validation
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)

svm_model = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
# evaluate model
scores = cross_val_score(model, X, Y, scoring='accuracy', cv=cv, n_jobs=-1)

# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores)*100, std(scores)))

Accuracy: 81.020 (0.008)


### **3. KNN**

In [32]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(data_final['text_final'],data_final['tag'],test_size=0.3)

In [33]:
Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [34]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data_final['text_final'])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [57]:
knn = KNeighborsClassifier()
#use gridsearch to test all values for n_neighbors
params_knn = {"n_neighbors": np.arange(1, 25)}

knn = GridSearchCV(knn, params_knn, cv=5)#fit model to training data

# fit the model
knn.fit(Train_X_Tfidf, Train_Y)

#save best model
knn_best = knn.best_estimator_#check best n_neigbors value
print(knn.best_params_)



{'n_neighbors': 4}


In [51]:
# Accuracy Score

knn = KNeighborsClassifier(n_neighbors = 4)
knn.fit(Train_X_Tfidf, Train_Y)
accuracy_train = accuracy_score(Train_Y, knn.predict(Train_X_Tfidf))
accuracy_test = accuracy_score(Test_Y, knn.predict(Test_X_Tfidf))

print ('Train_data Accuracy: %.2f' %(accuracy_train*100))
print ('Test_data Accuracy: %.2f' %(accuracy_test*100))

Train_data Accuracy: 74.02
Test_data Accuracy: 55.51


### 4. **Logistic Regression**

In [54]:
from sklearn.linear_model import LogisticRegression#create a new logistic regression model
log_reg = LogisticRegression()#fit the model to the training data
log_reg.fit(Train_X_Tfidf, Train_Y)
print("log_reg: {}".format(log_reg.score(Test_X_Tfidf, Test_Y)*100))

log_reg: 80.87485288348371


## **Ensemble**

In [67]:
from sklearn.ensemble import VotingClassifier

estimators=[("naive_model",naive_model),("log_reg",log_reg),("svm_model",svm_model)]
ensemble = VotingClassifier(estimators, voting="hard")

In [70]:
#fit model to training data
ensemble.fit(Train_X_Tfidf , Train_Y)#test our model on the test data
(ensemble.score(Test_X_Tfidf, Test_Y))*100

80.67869752844253